In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('universal_top_spotify_songs.csv')
# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 20

Типы данных по столбцам:
spotify_id           object
daily_rank            int64
daily_movement        int64
weekly_movement       int64
country              object
popularity            int64
is_explicit            bool
duration_ms           int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object


In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

data = data.drop('spotify_id', axis=1)

# Предположим, что мы хотим предсказать популярность песни
X = data.drop('popularity', axis=1)
y = data['popularity']
X = pd.get_dummies(X)

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение параметров для поиска
param_grid = {
    'iterations': [100, 200,],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6],
    'loss_function': ['MultiClass']
}

# Создание объекта GridSearchCV для модели
grid_search = GridSearchCV(
    estimator=CatBoostClassifier(verbose=False),  # Установка verbose=False,чтобы не было видно как обучается модель 
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)


# Подгонка модели к данным
grid_search.fit(X_train, y_train)

# Получение лучших параметров и оценщика с лучшими параметрами
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Получение прогнозов с использованием лучшей модели
y_pred = best_model.predict(X_test)

# Вычисление и вывод точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

# Оценка производительности модели
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

# Определение отсутствующих классов в прогнозах
classes = np.unique(y_test)
predicted_classes = np.unique(y_pred)


/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy v